In [ ]:
import sqlite3 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import cross_val_score
from collections import Counter
from sklearn.metrics import accuracy_score
from sklearn import cross_validation
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.metrics import confusion_matrix 
from sklearn import metrics 
from sklearn.metrics import roc_curve, auc 
from nltk.stem.porter import PorterStemmer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

- - - 

# Loading and Sampling the dataset(10k data-points)


In [0]:
final = pd.read_csv("final.csv")


final_data = final.sample(n = 10000)

final_data = final_data.drop(["Text"], axis = 1)
final_data = final_data.drop(final_data.columns[0], axis = 1)

In [5]:
labels = final_data.Score
final_data = final_data.sort_values("Time")
final_data.shape

(10000, 10)

# Train/Test Split

In [0]:
n = final_data.shape[0]
train_size = 0.7

train_set = final_data.iloc[:int(n*train_size)]
test_set =final_data.iloc[int(n*train_size):]

X_train = train_set.CleanedText
y_train = train_set.Score

X_test = test_set.CleanedText
y_test= test_set.Score

- - - 

# Bag of words Vectorization

In [28]:
count_vect = CountVectorizer() #in scikit-learn
X_train1 = count_vect.fit_transform(X_train)
X_test1 = count_vect.transform(X_test)

In [29]:
#Normalize
from sklearn.preprocessing import normalize
X_train1 = normalize(X_train1) 
X_test1  = normalize(X_test1) 



## GridSearch Cross-Validation

In [30]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC


Cs = [0.001, 0.01, 0.1, 1, 10]
gammas = [0.01, 0.1, 1, 10,100]
param_grid = {'C': Cs, 'gamma' : gammas}
rbf_svm = SVC(kernel='rbf')
rbf_svm_grid_cv = GridSearchCV(rbf_svm, param_grid=param_grid, n_jobs=-1, verbose=30)
rbf_svm_grid_cv

GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10], 'gamma': [0.01, 0.1, 1, 10, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=30)

In [31]:
#perform gridsearch
rbf_svm_grid_cv.fit(X_train1, y_train)

Fitting 3 folds for each of 25 candidates, totalling 75 fits
[CV] C=0.001, gamma=0.01 .............................................
[CV] C=0.001, gamma=0.01 .............................................
[CV] .... C=0.001, gamma=0.01, score=0.8474721508140531, total=   8.6s
[CV] C=0.001, gamma=0.01 .............................................


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   15.0s


[CV] .... C=0.001, gamma=0.01, score=0.8474721508140531, total=   9.2s
[CV] C=0.001, gamma=0.1 ..............................................


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   15.5s


[CV] ..... C=0.001, gamma=0.01, score=0.847770154373928, total=   9.7s
[CV] C=0.001, gamma=0.1 ..............................................


...

[CV] .......... C=10, gamma=10, score=0.847770154373928, total=  35.0s
[CV] C=10, gamma=100 .................................................


[Parallel(n_jobs=-1)]: Done  72 tasks      | elapsed: 18.7min


[CV] ........ C=10, gamma=100, score=0.8474721508140531, total=  35.4s
[CV] C=10, gamma=100 .................................................
[CV] ........ C=10, gamma=100, score=0.8474721508140531, total=  34.5s
[CV] ......... C=10, gamma=100, score=0.847770154373928, total=  34.5s


[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed: 19.9min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed: 19.9min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10], 'gamma': [0.01, 0.1, 1, 10, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=30)

In [33]:
print("Best parameters: ", rbf_svm_grid_cv.best_params_)
print("Best cross-validation score: {:.3f}".format(rbf_svm_grid_cv.best_score_))


Best parameters:  {'C': 10, 'gamma': 0.1}
Best cross-validation score: 0.896


In [35]:
from sklearn.metrics import classification_report

y_pred = rbf_svm_grid_cv.predict(X_test1)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

   negative       0.83      0.54      0.65       540
   positive       0.91      0.98      0.94      2460

avg / total       0.89      0.90      0.89      3000



## RandomSearch Cross-Validation

In [37]:
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats

param_grid  = {"C": stats.uniform(0.001, 10), "gamma": stats.uniform(0.1, 100)}

rbf_svm = SVC(kernel='rbf')
rbf_svm_grid_cv =  RandomizedSearchCV(rbf_svm, param_grid, n_jobs=-1, verbose=30)
rbf_svm_grid_cv


RandomizedSearchCV(cv=None, error_score='raise',
          estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
          fit_params=None, iid=True, n_iter=10, n_jobs=-1,
          param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff03c8a64e0>, 'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff03c8a6b38>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=30)

In [38]:
#perform gridsearch
rbf_svm_grid_cv.fit(X_train1, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] C=7.344875389757782, gamma=59.593754547849436 ...................
[CV] C=7.344875389757782, gamma=59.593754547849436 ...................
[CV]  C=7.344875389757782, gamma=59.593754547849436, score=0.8474721508140531, total=  15.2s
[CV] C=7.344875389757782, gamma=59.593754547849436 ...................


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   25.5s


[CV]  C=7.344875389757782, gamma=59.593754547849436, score=0.8474721508140531, total=  15.5s
[CV] C=1.4463249633789066, gamma=59.214645069168306 ..................


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   26.2s


[CV]  C=7.344875389757782, gamma=59.593754547849436, score=0.847770154373928, total=  15.8s
[CV] C=1.4463249633789066, gamma=59.214645069168306 ..................


...

[CV]  C=3.5333876179156607, gamma=50.005666866144104, score=0.847770154373928, total=  15.9s
[CV] C=1.347251919682676, gamma=60.17423447209802 ....................


[Parallel(n_jobs=-1)]: Done  27 tasks      | elapsed:  5.6min


[CV]  C=1.347251919682676, gamma=60.17423447209802, score=0.8474721508140531, total=  15.6s
[CV] C=1.347251919682676, gamma=60.17423447209802 ....................
[CV]  C=1.347251919682676, gamma=60.17423447209802, score=0.8474721508140531, total=  15.1s
[CV]  C=1.347251919682676, gamma=60.17423447209802, score=0.847770154373928, total=  15.4s


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  6.1min finished


RandomizedSearchCV(cv=None, error_score='raise',
          estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
          fit_params=None, iid=True, n_iter=10, n_jobs=-1,
          param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff03c8a64e0>, 'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff03c8a6b38>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=30)

In [39]:
print("Best parameters: ", rbf_svm_grid_cv.best_params_)
print("Best cross-validation score: {:.3f}".format(rbf_svm_grid_cv.best_score_))


Best parameters:  {'C': 1.520133118164242, 'gamma': 0.42656776863930757}
Best cross-validation score: 0.889


In [40]:
y_pred = rbf_svm_grid_cv.predict(X_test1)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

   negative       0.90      0.41      0.56       540
   positive       0.88      0.99      0.93      2460

avg / total       0.89      0.89      0.87      3000



- - - 

# TF-idf Vectorization

In [0]:
#TF-IDF
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2))
X_train2 = tf_idf_vect.fit_transform(X_train)
X_test2 = tf_idf_vect.transform(X_test)

In [0]:
#Normalize
from sklearn.preprocessing import normalize
X_train2 = normalize(X_train2) 
X_test2  = normalize(X_test2) 



## GridSearch Cross-Validation

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC


Cs = [0.001, 0.01, 0.1, 1, 10]
gammas = [0.01, 0.1, 1, 10,100]
param_grid = {'C': Cs, 'gamma' : gammas}
rbf_svm = SVC(kernel='rbf')
rbf_svm_grid_cv = GridSearchCV(rbf_svm, param_grid=param_grid, n_jobs = -1, verbose=30)
rbf_svm_grid_cv

#perform gridsearch
rbf_svm_grid_cv.fit(X_train2, y_train)

Fitting 3 folds for each of 25 candidates, totalling 75 fits
[CV] C=0.001, gamma=0.01 .............................................
[CV] C=0.001, gamma=0.01 .............................................
[CV] .... C=0.001, gamma=0.01, score=0.8534076296613802, total=   5.5s
[CV] C=0.001, gamma=0.01 .............................................


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    9.3s


[CV] .... C=0.001, gamma=0.01, score=0.8534704370179949, total=   5.6s
[CV] C=0.001, gamma=0.1 ..............................................


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    9.5s


[CV] .... C=0.001, gamma=0.01, score=0.8534076296613802, total=   5.4s
[CV] C=0.001, gamma=0.1 ..............................................


...

[CV] ......... C=10, gamma=10, score=0.8534076296613802, total=  17.5s
[CV] C=10, gamma=100 .................................................


[Parallel(n_jobs=-1)]: Done  72 tasks      | elapsed: 12.4min


[CV] ........ C=10, gamma=100, score=0.8534704370179949, total=  17.6s
[CV] C=10, gamma=100 .................................................
[CV] ........ C=10, gamma=100, score=0.8534076296613802, total=  17.5s
[CV] ........ C=10, gamma=100, score=0.8534076296613802, total=  16.7s


[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed: 13.1min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed: 13.1min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10], 'gamma': [0.01, 0.1, 1, 10, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=30)

In [11]:
print("Best parameters: ", rbf_svm_grid_cv.best_params_)
print("Best cross-validation score: {:.3f}".format(rbf_svm_grid_cv.best_score_))


Best parameters:  {'C': 10, 'gamma': 0.1}
Best cross-validation score: 0.894


In [12]:
from sklearn.metrics import classification_report

y_pred = rbf_svm_grid_cv.predict(X_test2)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

   negative       0.83      0.52      0.64       569
   positive       0.90      0.98      0.93      2431

avg / total       0.88      0.89      0.88      3000



## RandomSearch Cross-Validation

In [13]:
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats

param_grid  = {"C": stats.uniform(0.001, 10), "gamma": stats.uniform(0.1, 100)}

rbf_svm = SVC(kernel='rbf')
rbf_svm_grid_cv =  RandomizedSearchCV(rbf_svm, param_grid, n_jobs=-1, verbose=30)
rbf_svm_grid_cv

#perform gridsearch
rbf_svm_grid_cv.fit(X_train2, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] C=3.9642771832596115, gamma=29.78300445438972 ...................
[CV] C=3.9642771832596115, gamma=29.78300445438972 ...................
[CV]  C=3.9642771832596115, gamma=29.78300445438972, score=0.8534076296613802, total=  17.9s
[CV] C=3.9642771832596115, gamma=29.78300445438972 ...................


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   29.9s


[CV]  C=3.9642771832596115, gamma=29.78300445438972, score=0.8534704370179949, total=  18.2s
[CV] C=1.3456356932942815, gamma=38.7452364654463 ....................


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   30.6s


[CV]  C=3.9642771832596115, gamma=29.78300445438972, score=0.8534076296613802, total=  18.0s
[CV] C=1.3456356932942815, gamma=38.7452364654463 ....................


...

[CV]  C=5.615261329734922, gamma=66.67437251896835, score=0.8534076296613802, total=  17.4s
[CV] C=9.710920645923661, gamma=65.1088261351427 .....................


[Parallel(n_jobs=-1)]: Done  27 tasks      | elapsed:  6.9min


[CV]  C=9.710920645923661, gamma=65.1088261351427, score=0.8534704370179949, total=  17.4s
[CV] C=9.710920645923661, gamma=65.1088261351427 .....................
[CV]  C=9.710920645923661, gamma=65.1088261351427, score=0.8534076296613802, total=  17.3s
[CV]  C=9.710920645923661, gamma=65.1088261351427, score=0.8534076296613802, total=  16.9s


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  7.4min finished


RandomizedSearchCV(cv=None, error_score='raise',
          estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
          fit_params=None, iid=True, n_iter=10, n_jobs=-1,
          param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fddf9f40780>, 'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fddf9f409b0>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=30)

In [14]:
print("Best parameters: ", rbf_svm_grid_cv.best_params_)
print("Best cross-validation score: {:.3f}".format(rbf_svm_grid_cv.best_score_))


Best parameters:  {'C': 3.9642771832596115, 'gamma': 29.78300445438972}
Best cross-validation score: 0.853


In [15]:
y_pred = rbf_svm_grid_cv.predict(X_test2)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

   negative       0.00      0.00      0.00       569
   positive       0.81      1.00      0.90      2431

avg / total       0.66      0.81      0.73      3000



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.



- - - 

# Word2Vec Vectorization

In [16]:
import nltk
nltk.download('stopwords')
import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

stop = set(stopwords.words('english')) #set of stopwords
sno = nltk.stem.SnowballStemmer('english') #initialising the snowball stemmer

def cleanhtml(sentence): #function to clean the word of any html-tags
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext
def cleanpunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned
print(stop)
print('************************************')
print(sno.stem('tasty'))

[nltk_data] Downloading package stopwords to /content/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
{"it's", 'but', 'how', 'both', 'such', 'yourself', 'before', 'hasn', 'their', 'be', 'over', 'just', "couldn't", 'doesn', 'were', "should've", 'down', 'about', 're', 'won', "hadn't", 'myself', 'or', 'your', "wouldn't", 'why', 'whom', 'again', 'has', 'being', 'couldn', 'aren', "needn't", 'isn', 'a', 'shan', 'on', 'was', 'yourselves', 'ain', 'didn', 'i', 'yours', 'of', 'so', 'after', "isn't", 'it', 'not', 'until', 'needn', 'ours', 'above', "mustn't", 'out', 'some', 'most', "aren't", 'having', 'doing', 'an', 'y', 'too', 'will', "didn't", "mightn't", 'hers', 'as', 'own', 'do', 'if', 'weren', 'from', 'his', 'are', 'wouldn', 'because', 'into', 'for', 'where', "you'd", "you've", 'them', 'mightn', 'd', 'did', 'ourselves', 'they', 'haven', "weren't", 'to', 'herself', 'himself', 'all', 'mustn', 'o', 'by', 'during', 'what', 'no', 'ma', 'shouldn', 'then', 'themselves', 'in', "wasn't", '

##  Training Word2Vec model using own text corpus






In [0]:
import gensim
i=0
train_sent=[]
for sent in X_train:
    filtered_sentence=[]
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(cleaned_words.isalpha()):    
                filtered_sentence.append(cleaned_words.lower())
            else:
                continue 
    train_sent.append(filtered_sentence)
    

In [0]:
test_sent=[]
for sent in X_test:
    filtered_sentence=[]
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(cleaned_words.isalpha()):    
                filtered_sentence.append(cleaned_words.lower())
            else:
                continue 
    test_sent.append(filtered_sentence)

In [0]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

w2v_model=gensim.models.Word2Vec(train_sent,min_count=5,size=50, workers=4)    


- - - 

## Applying Average Word2vec

In [23]:
#AVG-W2V
sent_vectors = []; # the avg-w2v for each sentence/review is stored in this list
for sent in train_sent: # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        try:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
        except:
            pass
    sent_vec /= cnt_words
    sent_vectors.append(sent_vec)
print(len(sent_vectors))
print(len(sent_vectors[0]))

7000
50


In [24]:
#AVG-W2V
sent_vectors2 = []; # the avg-w2v for each sentence/review is stored in this list
for sent in test_sent: # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        try:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
        except:
            pass
    sent_vec /= cnt_words
    sent_vectors2.append(sent_vec)
print(len(sent_vectors2))
print(len(sent_vectors2[0]))

3000
50


In [0]:
#Normalize
from sklearn.preprocessing import normalize
X_train3 = normalize(sent_vectors) 
X_test3 = normalize(sent_vectors2) 

## Applying GridSearchCV

In [26]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC


Cs = [0.001, 0.01, 0.1, 1, 10]
gammas = [0.01, 0.1, 1, 10,100]
param_grid = {'C': Cs, 'gamma' : gammas}
rbf_svm = SVC(kernel='rbf')
rbf_svm_grid_cv = GridSearchCV(rbf_svm, param_grid=param_grid, n_jobs=-1, verbose=30)
rbf_svm_grid_cv

#perform gridsearch
rbf_svm_grid_cv.fit(X_train3, y_train)

Fitting 3 folds for each of 25 candidates, totalling 75 fits
[CV] C=0.001, gamma=0.01 .............................................
[CV] C=0.001, gamma=0.01 .............................................
[CV] .... C=0.001, gamma=0.01, score=0.8534076296613802, total=   0.8s
[CV] C=0.001, gamma=0.01 .............................................
[CV] .... C=0.001, gamma=0.01, score=0.8534704370179949, total=   0.9s
[CV] C=0.001, gamma=0.1 ..............................................


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.4s


[CV] .... C=0.001, gamma=0.01, score=0.8534076296613802, total=   1.1s
[CV] C=0.001, gamma=0.1 ..............................................
[CV] ..... C=0.001, gamma=0.1, score=0.8534704370179949, total=   1.2s
[CV] C=0.001, gamma=0.1 ..............................................


[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    3.4s


[CV] ..... C=0.001, gamma=0.1, score=0.8534076296613802, total=   1.2s
[CV] C=0.001, gamma=1 ................................................
[CV] ..... C=0.001, gamma=0.1, score=0.8534076296613802, total=   1.2s
[CV] C=0.001, gamma=1 ................................................


...

[CV] .......... C=10, gamma=10, score=0.853836262323189, total=   2.1s
[CV] C=10, gamma=100 .................................................


[Parallel(n_jobs=-1)]: Done  72 tasks      | elapsed:  1.6min


[CV] ........ C=10, gamma=100, score=0.8273350471293917, total=   3.4s
[CV] C=10, gamma=100 .................................................
[CV] ........ C=10, gamma=100, score=0.8251178739819974, total=   3.4s
[CV] ........ C=10, gamma=100, score=0.8251178739819974, total=   3.1s


[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:  1.7min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:  1.7min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10], 'gamma': [0.01, 0.1, 1, 10, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=30)

In [27]:
print("Best parameters: ", rbf_svm_grid_cv.best_params_)
print("Best cross-validation score: {:.3f}".format(rbf_svm_grid_cv.best_score_))


Best parameters:  {'C': 10, 'gamma': 10}
Best cross-validation score: 0.856


In [28]:
from sklearn.metrics import classification_report

y_pred = rbf_svm_grid_cv.predict(X_test3)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

   negative       0.60      0.04      0.08       569
   positive       0.82      0.99      0.90      2431

avg / total       0.77      0.81      0.74      3000



---

## RandomSearch Cross-Validation

In [29]:
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats

param_grid  = {"C": stats.uniform(0.001, 10), "gamma": stats.uniform(0.1, 100)}

rbf_svm = SVC(kernel='rbf')
rbf_svm_grid_cv =  RandomizedSearchCV(rbf_svm, param_grid, n_jobs=-1, verbose=30)
rbf_svm_grid_cv

#perform gridsearch
rbf_svm_grid_cv.fit(X_train3, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] C=4.831589679522305, gamma=17.36777007238206 ....................
[CV] C=4.831589679522305, gamma=17.36777007238206 ....................
[CV]  C=4.831589679522305, gamma=17.36777007238206, score=0.8572653236176596, total=   1.7s
[CV] C=4.831589679522305, gamma=17.36777007238206 ....................
[CV]  C=4.831589679522305, gamma=17.36777007238206, score=0.8551842330762639, total=   1.8s
[CV] C=3.555084986226801, gamma=57.56056193440211 ....................


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.8s


[CV]  C=4.831589679522305, gamma=17.36777007238206, score=0.8534076296613802, total=   2.4s
[CV] C=3.555084986226801, gamma=57.56056193440211 ....................


[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    6.1s


[CV]  C=3.555084986226801, gamma=57.56056193440211, score=0.8526135389888603, total=   2.8s
[CV] C=3.555084986226801, gamma=57.56056193440211 ....................


...

[CV]  C=6.8506275628906215, gamma=38.017778693382155, score=0.8439777111015859, total=   2.4s
[CV] C=2.8135405635998967, gamma=83.68330581001806 ...................


[Parallel(n_jobs=-1)]: Done  27 tasks      | elapsed:   51.9s


[CV]  C=2.8135405635998967, gamma=83.68330581001806, score=0.8496143958868895, total=   2.8s
[CV] C=2.8135405635998967, gamma=83.68330581001806 ...................
[CV]  C=2.8135405635998967, gamma=83.68330581001806, score=0.8431204457779683, total=   2.7s
[CV]  C=2.8135405635998967, gamma=83.68330581001806, score=0.8474067723960566, total=   2.4s


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   57.7s finished


RandomizedSearchCV(cv=None, error_score='raise',
          estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
          fit_params=None, iid=True, n_iter=10, n_jobs=-1,
          param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fddf4dea0f0>, 'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fddf442b358>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=30)

In [30]:
print("Best parameters: ", rbf_svm_grid_cv.best_params_)
print("Best cross-validation score: {:.3f}".format(rbf_svm_grid_cv.best_score_))


Best parameters:  {'C': 4.831589679522305, 'gamma': 17.36777007238206}
Best cross-validation score: 0.855


In [31]:
y_pred = rbf_svm_grid_cv.predict(X_test3)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

   negative       0.58      0.04      0.07       569
   positive       0.82      0.99      0.90      2431

avg / total       0.77      0.81      0.74      3000



- - - 

# Word2Vec Tf-idf Vectorization


In [0]:
#TF-IDF

tf_idf_vect = TfidfVectorizer()
final_tf_idf = tf_idf_vect.fit_transform(X_train)
tfidf_feat = tf_idf_vect.get_feature_names()
 # tfidf words/col-names
# final_tf_idf is the sparse matrix with row= sentence, col=word and cell_val = tfidf

train_vectors = []; # the tfidf-w2v for each sentence/review is stored in this list
row=0;
for sent in train_sent: # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length
    weight_sum = 0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        try:
            vec = w2v_model.wv[word]
            # obtain the tf_idf of a word in a sentence/review
            tfidf = final_tf_idf[row, tfidf_feat.index(word)]
            sent_vec += (vec * tfidf)
            weight_sum += tfidf
            
            
        except:
            pass

    sent_vec /= weight_sum
    #print(np.isnan(np.sum(sent_vec)))

    train_vectors.append(sent_vec)
    row += 1

In [0]:
#TF-IDF

tf_idf_vect = TfidfVectorizer()
final_tf_idf = tf_idf_vect.fit_transform(X_test)
tfidf_feat = tf_idf_vect.get_feature_names()
 # tfidf words/col-names
# final_tf_idf is the sparse matrix with row= sentence, col=word and cell_val = tfidf

test_vectors = []; # the tfidf-w2v for each sentence/review is stored in this list
row=0;
for sent in test_sent: # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length
    weight_sum = 0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        try:
            vec = w2v_model.wv[word]
            # obtain the tf_idf of a word in a sentence/review
            tfidf = final_tf_idf[row, tfidf_feat.index(word)]
            sent_vec += (vec * tfidf)
            weight_sum += tfidf
            
            
        except:
            pass

    sent_vec /= weight_sum
    #print(np.isnan(np.sum(sent_vec)))

    test_vectors.append(sent_vec)
    row += 1

In [0]:
#Normalize
from sklearn.preprocessing import normalize
X_train4 = normalize(train_vectors) 
X_test4 = normalize(test_vectors) 

## Applying GridSearchCV

In [37]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC


Cs = [0.001, 0.01, 0.1, 1, 10]
gammas = [0.01, 0.1, 1, 10,100]
param_grid = {'C': Cs, 'gamma' : gammas}
rbf_svm = SVC(kernel='rbf')
rbf_svm_grid_cv = GridSearchCV(rbf_svm, param_grid=param_grid, n_jobs=-1, verbose=30)
rbf_svm_grid_cv

#perform gridsearch
rbf_svm_grid_cv.fit(X_train4, y_train)

Fitting 3 folds for each of 25 candidates, totalling 75 fits
[CV] C=0.001, gamma=0.01 .............................................
[CV] C=0.001, gamma=0.01 .............................................
[CV] .... C=0.001, gamma=0.01, score=0.8534704370179949, total=   0.9s
[CV] .... C=0.001, gamma=0.01, score=0.8534076296613802, total=   0.9s
[CV] C=0.001, gamma=0.01 .............................................
[CV] C=0.001, gamma=0.1 ..............................................


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.4s


[CV] .... C=0.001, gamma=0.01, score=0.8534076296613802, total=   1.0s
[CV] ..... C=0.001, gamma=0.1, score=0.8534704370179949, total=   1.0s
[CV] C=0.001, gamma=0.1 ..............................................
[CV] C=0.001, gamma=0.1 ..............................................


[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    3.1s


[CV] ..... C=0.001, gamma=0.1, score=0.8534076296613802, total=   1.2s
[CV] C=0.001, gamma=1 ................................................
[CV] ..... C=0.001, gamma=0.1, score=0.8534076296613802, total=   1.2s
[CV] C=0.001, gamma=1 ................................................


...

[CV] ......... C=10, gamma=10, score=0.8542648949849978, total=   2.3s
[CV] C=10, gamma=100 .................................................


[Parallel(n_jobs=-1)]: Done  72 tasks      | elapsed:  1.6min


[CV] ........ C=10, gamma=100, score=0.8234790059982862, total=   3.7s
[CV] C=10, gamma=100 .................................................
[CV] ......... C=10, gamma=100, score=0.821260180025718, total=   3.6s
[CV] ........ C=10, gamma=100, score=0.8169738534076296, total=   3.2s


[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:  1.7min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:  1.7min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10], 'gamma': [0.01, 0.1, 1, 10, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=30)

In [38]:
print("Best parameters: ", rbf_svm_grid_cv.best_params_)
print("Best cross-validation score: {:.3f}".format(rbf_svm_grid_cv.best_score_))


Best parameters:  {'C': 1, 'gamma': 100}
Best cross-validation score: 0.854


In [39]:
from sklearn.metrics import classification_report

y_pred = rbf_svm_grid_cv.predict(X_test4)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

   negative       0.73      0.01      0.03       569
   positive       0.81      1.00      0.90      2431

avg / total       0.80      0.81      0.73      3000



- - - 

## Applying RandomSearch CV

In [40]:
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats

param_grid  = {"C": stats.uniform(0.001, 10), "gamma": stats.uniform(0.1, 100)}

rbf_svm = SVC(kernel='rbf')
rbf_svm_grid_cv =  RandomizedSearchCV(rbf_svm, param_grid, n_jobs=-1, verbose=30)
rbf_svm_grid_cv

#perform gridsearch
rbf_svm_grid_cv.fit(X_train4, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] C=3.826310675830332, gamma=38.987418035791514 ...................
[CV] C=3.826310675830332, gamma=38.987418035791514 ...................
[CV]  C=3.826310675830332, gamma=38.987418035791514, score=0.8495499357051007, total=   1.8s
[CV] C=3.826310675830332, gamma=38.987418035791514 ...................
[CV]  C=3.826310675830332, gamma=38.987418035791514, score=0.8419023136246787, total=   1.8s
[CV] C=2.037069535239874, gamma=55.27804816327809 ....................


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.8s


[CV]  C=3.826310675830332, gamma=38.987418035791514, score=0.8448349764252036, total=   2.7s
[CV] C=2.037069535239874, gamma=55.27804816327809 ....................


[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    6.7s


[CV]  C=2.037069535239874, gamma=55.27804816327809, score=0.8470437017994858, total=   2.7s
[CV] C=2.037069535239874, gamma=55.27804816327809 ....................


...

[CV]  C=7.082857219471582, gamma=8.584740676392588, score=0.8534076296613802, total=   2.2s
[CV] C=6.469737291475392, gamma=8.49642449206237 .....................


[Parallel(n_jobs=-1)]: Done  27 tasks      | elapsed:   49.2s


[CV]  C=6.469737291475392, gamma=8.49642449206237, score=0.8534704370179949, total=   2.1s
[CV] C=6.469737291475392, gamma=8.49642449206237 .....................
[CV]  C=6.469737291475392, gamma=8.49642449206237, score=0.8534076296613802, total=   2.2s
[CV]  C=6.469737291475392, gamma=8.49642449206237, score=0.8534076296613802, total=   2.1s


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   53.1s finished


RandomizedSearchCV(cv=None, error_score='raise',
          estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
          fit_params=None, iid=True, n_iter=10, n_jobs=-1,
          param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fddf8efe5c0>, 'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fddf8ef84e0>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=30)

In [41]:
print("Best parameters: ", rbf_svm_grid_cv.best_params_)
print("Best cross-validation score: {:.3f}".format(rbf_svm_grid_cv.best_score_))


Best parameters:  {'C': 7.696926304605969, 'gamma': 1.9372036764791933}
Best cross-validation score: 0.853


- - -

# Conclusion

### The best results are obtained using 'Bag of words' vectorization technique with hyperparameter C = 10,and <br>         gamma = 0.1, the best CV score achieved is 89.6%.